In [27]:
import torch
from torch import nn
from tqdm import tqdm
import torchvision.transforms as transforms
import torchvision.datasets as datasets
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [28]:
training_config = {'BATCH_SIZE' : 1024,
                   'lr' : 1e-4,
                   'epochs' : 100,
                   'Experiment_name': 'Exp4'
                   }

test_dataset = datasets.CIFAR10(root='./data', train=False, transform=transforms.ToTensor(), download=True)

eval_loader =  torch.utils.data.DataLoader(
        dataset=test_dataset,
        batch_size=training_config['BATCH_SIZE'],
        shuffle=False,
        drop_last=True
    )

Files already downloaded and verified


In [29]:
def evaluate (model : nn.Module, dataloader ):
    print("##### EVALUATION ... ####")
    model.eval()
    n_correct = 0

    with torch.no_grad():
        progress_bar = tqdm(enumerate(dataloader), total=len(dataloader))
        for i, (imgs, labels) in progress_bar: 
            imgs, labels = imgs.to(device), labels.to(device)

            if model == model_1 or model == model_2:
                flattened_imgs = imgs.flatten(start_dim=1)
                preds = model(flattened_imgs)
            else:
                preds = model(imgs)

            pred_labels = torch.argmax(torch.softmax(preds, dim=-1), dim=-1)
            cur_correct = len(torch.where(pred_labels == labels)[0])
            n_correct = n_correct + cur_correct

    accuracy = n_correct / len(test_dataset) * 100
    print(f"Test accuracy: {round(accuracy,2)}%") 

# _____________________________________________________________________________________________________________________
# <center>**Analysis and Comparison**</center>
# 
# 
# 

##  **MLP**

#### Haha we changed the code for model architecture to go from experiment 1 to experiment 2 and the architecture for experiment 1 is not saved anywhere; the weights are though. For reconstructing the model for experiment 1, I will proceed by guessing it from layer dimensions.

### **Experiment 1**:

In [32]:
checkpoints_1 = torch.load("./models/Exp4/checkpoint_Exp4.pth")

for key in checkpoints_1['model_state_dict'].keys():
    print(key, checkpoints_1['model_state_dict'][key].shape)

network.0.weight torch.Size([32, 3, 3, 3])
network.0.bias torch.Size([32])
network.2.weight torch.Size([64, 32, 3, 3])
network.2.bias torch.Size([64])
network.5.weight torch.Size([128, 64, 3, 3])
network.5.bias torch.Size([128])
network.7.weight torch.Size([128, 128, 3, 3])
network.7.bias torch.Size([128])
network.10.weight torch.Size([256, 128, 3, 3])
network.10.bias torch.Size([256])
network.12.weight torch.Size([256, 256, 3, 3])
network.12.bias torch.Size([256])
network.16.weight torch.Size([1024, 4096])
network.16.bias torch.Size([1024])
network.18.weight torch.Size([512, 1024])
network.18.bias torch.Size([512])
network.20.weight torch.Size([10, 512])
network.20.bias torch.Size([10])


#### Ok clear now. And layer 1 is ReLu so no weights and biases there. For experiment 1, I will have to reconstruct its MLP class.

In [ ]:
state_dict = checkpoints_1['model_state_dict']

In [ ]:
class MLP1(nn.Module):
    """
    MLP composed of two fully connected layers.
     - First layer takes pixel values and maps them to a hidden dimension
     - Nonlinear activation
     - Second layer maps from hidden dimension to number of classes, predicting a score for each of the classes
    """
    def __init__(self, input_dim=3072, hidden_dim=1024, output_dim=10):
        """ Model initalizer """
        super().__init__()
        self.layers = nn.Sequential(
                nn.Linear(in_features=input_dim, out_features=hidden_dim),
                nn.ReLU(),
                nn.Linear(in_features=hidden_dim, out_features=output_dim)
            )
        
    def forward(self, x):
        """ Forward pass through the model"""
        assert len(x.shape) == 2, f"ERROR! Shape of input must be 2D (b_size, dim)"
        pred = self.layers(x)
        return pred

In [ ]:
model_1 = MLP1().to(device)
model_1.load_state_dict(state_dict=state_dict)

<All keys matched successfully>

In [ ]:
evaluate(model= model_1, dataloader= eval_loader)

##### EVALUATION ... ####


100%|██████████| 9/9 [00:00<00:00, 14.52it/s]

Test accuracy: 53844.44%


### **Experiment 2:**

## **CONV**